In [ ]:
%pip install transformers

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium").to(device)



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
#code to test current initial model from the dialoGPT documentation https://huggingface.co/microsoft/DialoGPT-medium?text=Hey+my+name+is+Julien%21+How+are+you%3F
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:hello


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Hello! :D
>> User:I have an airline complaint that my leg space was not good


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I have an airline complaint that my leg space was not good
>> User:hi


...

DialoGPT: Hi! :D
>> User:sup


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Sup :D
>> User:qwer


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Qwer qwer


In [ ]:
import pandas as pd

file_path = '/content/drive/My Drive/cs182 final proj/CodeForCollab/data182/Conversation.csv'
data = pd.read_csv(file_path)

In [ ]:
data['paired'] = data['question'].astype(str) + tokenizer.eos_token + data['answer'].astype(str)

tokenizer.pad_token = tokenizer.eos_token
tokenized_data = tokenizer(data['paired'].tolist(), padding=True, truncation=True, return_tensors="pt")

In [ ]:
import torch
import torch.nn as nn

class CustomLossForAirline(nn.Module):
    def __init__(self):
        super(CustomLossForAirline, self).__init__()

    def forward(self, predictions, targets):
        standard_loss = nn.CrossEntropyLoss()(predictions, targets)
        custom_criteria_loss = self.calculate_custom_loss(predictions)
        total_loss = standard_loss + custom_criteria_loss

        return total_loss

    def calculate_custom_loss(self, predictions):
        tooshort = 3
        toolong = 100
        custom_loss_component = 0
        unwanted_terms = ['terrible','ugly','worst', 'stupid', 'ridiculous']

        probabilities = torch.softmax(predictions, dim=-1)
        unwanted_indices = [tokenizer.vocab[word] for word in unwanted_terms if word in tokenizer.vocab]
        unwanted_probs = probabilities[:, unwanted_indices].sum()

        scale_factor = 100
        custom_loss_component += scale_factor * unwanted_probs

        if len(predictions) <= tooshort:
            custom_loss_component = 100
        elif len(predictions) >= toolong:
            custom_loss_component = 100

        return custom_loss_component

In [ ]:
%pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
%pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.8 MB/s eta 0:00:00


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./model_output",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()

ImportError: ignored